In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time
import pandas as pd

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.mars_db
collection = db.articles

In [4]:
# Nasa Mars News
final_scrape_output ={}

In [5]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [6]:
# Retrieve the parent divs for all articles
news_title = soup.find('div', class_='content_title').text.strip()

final_scrape_output.update(
    {'news_title': news_title}
)

print(news_title)

10.9 Million Names Now Aboard NASA's Perseverance Mars Rover


In [7]:
news_p = soup.find('div', class_='rollover_description_inner').text.strip()

print(news_p)


As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars.


In [8]:
#JPL Mars Space Images - Featured Image

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# URL of page to be scraped
url_img = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(url_img)

try:
    browser.select('category', 'featured')
    bg_attribute_string =browser.find_by_css('article').first['style']
    bg_attribute_string_left = bg_attribute_string.split('("',1)[1]
    bg_attribute_string_final = bg_attribute_string_left.split('")',1)[0]
    
except ElementDoesNotExist:
    print("Clicking Complete")

print(bg_attribute_string_final)

UnboundLocalError: local variable 'e' referenced before assignment

In [9]:
featured_image_url = 'https://www.jpl.nasa.gov' + bg_attribute_string_final

print(featured_image_url)

NameError: name 'bg_attribute_string_final' is not defined

In [10]:
# Mars Weather

# URL of page to be scraped
url_weather = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response_weather = requests.get(url_weather)

# Create BeautifulSoup object; parse with 'lxml'
soup_weather = BeautifulSoup(response_weather.text, 'lxml')

#Extract text from tweet
weather_p = soup_weather.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text.strip()

#Clean up text from tweet
mars_weather = weather_p.replace('\n',', ')

In [11]:
print(mars_weather)

If you submitted your name, it’s aboard the Perseverance Mars Rover nowhttps://mars.nasa.gov/news/8634/109-million-names-now-aboard-nasas-perseverance-mars-rover …


In [12]:
# Mars Facts

url_facts = 'https://space-facts.com/mars/'

# Use Panda's `read_html` to parse the url
tables = pd.read_html(url_facts)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [13]:
#Load data into DataFrame
facts_df = tables[0]
facts_df.rename(columns={
    0:"Description",
    1:"Value"
}, inplace=True)
facts_df.set_index('Description',inplace=True)
facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [14]:
#convert pandas into html table
facts_df.to_html('mars_facts_table.html')

facts_html = facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [15]:
# Mars Hemispheres

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
url_mars_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url_mars_hemispheres)

#loop to get all mars hemispheres
hemisphere_image_urls = []

mars_hemishpheres = [
    "Valles Marineris Hemisphere",
    "Cerberus Hemisphere",
    "Schiaparelli Hemisphere",
    "Syrtis Major Hemisphere"
]

for hemisphere in mars_hemishpheres:
    hemisphere_image_urls_dict = {}
    browser.find_by_text(f'{hemisphere} Enhanced').click()
    img_url_href = browser.find_by_text('Sample')['href']
    hemisphere_image_urls_dict.update(
        {'title' : hemisphere,
        'img_url': img_url_href
        }
    )
    hemisphere_image_urls.append(hemisphere_image_urls_dict)
    browser.back()
    
print(hemisphere_image_urls)

UnboundLocalError: local variable 'e' referenced before assignment